In [ ]:
# 下記セルを実行すると、Googleドライブへの接続を求められます。
# Googleアカウントにログインして進めてください。
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
path = f'/content/drive/MyDrive/{working_dir}/本章/放課後'
os.chdir(path)

# 放課後_大規模言語モデル（LLM）を活用した２０本ノック



### 放課後ノック102： APIを用いてLLMを使ってみよう

In [ ]:
!pip install openai==1.26.0

In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "面白い挨拶を幾つか教えて",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0
)
print(chat_completion)

In [ ]:
print(chat_completion.choices[0].message.content)

### 放課後ノック103： モデルやランダム性を変えて比べてみよう

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "面白い挨拶を幾つか教えて",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0.8
)
print(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "面白い挨拶を幾つか教えて",
        }
    ],
    model="gpt-4",
    temperature=0
)
print(chat_completion.choices[0].message.content)

### 放課後ノック104： 長文での指示を工夫してみよう

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "今日は、10時から11時までWebで会議をした後に、移動を行い、対面で2時間の会議を行う予定でしたが、1時間で終わりました。そのため、30分の短い会議を社内で行いました。今日の会議時間の合計は？",
        }
    ],
    model="gpt-4",
    temperature=0
)
print(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "今日は、10時から11時までWebで会議をした後に、移動を行い、対面で2時間の会議を行う予定でしたが、1時間で終わりました。そのため、30分の短い会議を社内で行いました。今日の会議時間の合計は？ １つずつ順番に考えてみましょう。",
        }
    ],
    model="gpt-4",
    temperature=0
)
print(chat_completion.choices[0].message.content)

### 放課後ノック105： 質問の履歴を確認してみよう

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "こんにちは。私の名前はマイクです。",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0
)
print(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "私の名前が分かりますか？",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0
)
print(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "こんにちは。私の名前はマイクです。"}, # 1つ目の質問
        {"role": "assistant", "content": "こんにちは、マイクさん。どうぞよろしくお願いします。何かお手伝いできることがあればお知らせくださいね。"}, # 1つ目の質問の回答
        {"role": "user", "content": "私の名前が分かりますか？"} # 2つ目の質問
    ],
    model="gpt-3.5-turbo",
    temperature=0
)
print(chat_completion.choices[0].message.content)

### 放課後ノック106： Langchainを使ってみよう

In [ ]:
!pip install langchain==0.1.17
!pip install langchain-openai==0.1.6

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0,
                  api_key=os.environ.get("OPENAI_API_KEY"))

messages = [
     HumanMessage(content="こんにちは。私の名前はマイクです。"), # 1つ目の質問
     AIMessage(content="こんにちは、マイクさん。どうぞよろしくお願いします。何かお手伝いできることがあればお知らせくださいね。"), # 1つ目の質問の回答
     HumanMessage(content= "私の名前が分かりますか？") # 2つ目の質問
 ]

res = chat.invoke(messages)
res

In [ ]:
print(res.content)

### 放課後ノック107： 日本語を外国語に翻訳してみよう

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは日本語を{language}に訳す翻訳家です。"),
    HumanMessagePromptTemplate.from_template("以下の文章を翻訳してください。文章: {text}")
 ])

In [ ]:
messages = chat_prompt.format_prompt(language="中国語", text="こんにちは").to_messages()
print(messages)

In [ ]:
res = chat.invoke(messages)
print(res.content)

In [ ]:
messages = chat_prompt.format_prompt(language="韓国語", text="こんにちは").to_messages()
res = chat.invoke(messages)
print(res.content)

### 放課後ノック108： 応答内容をjsonや辞書型で出力してみよう

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="原文", description="原文"),
    ResponseSchema(name="翻訳結果", description="翻訳結果")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは日本語を{language}に訳す翻訳家です。{format_instructions}"),
    HumanMessagePromptTemplate.from_template("以下の文章を翻訳してください。文章: {text}")
 ])

messages = chat_prompt.format_prompt(language="韓国語", text="こんにちは", format_instructions=format_instructions).to_messages()
res = chat.invoke(messages)
print(res.content)

In [ ]:
res_dict = output_parser.parse(res.content)
res_dict

In [ ]:
print(res_dict['原文'])
print(res_dict['翻訳結果'])

### 放課後ノック109： 応答内容をリストで取得してみよう

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("{format_instructions}"),
    HumanMessagePromptTemplate.from_template("面白い挨拶を幾つか教えて")
 ])

messages = chat_prompt.format_prompt(format_instructions=format_instructions).to_messages()
res = chat.invoke(messages)
print(res.content)

In [ ]:
res_list = output_parser.parse(res.content)
print(res_list)
print(type(res_list))

### 放課後ノック110： 翻訳システムを意識してみよう

In [ ]:
response_schemas = [
    ResponseSchema(name="原文", description="原文"),
    ResponseSchema(name="翻訳結果", description="翻訳結果"),
    ResponseSchema(name="発音", description="翻訳結果の日本語での発音")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
language = '韓国語'
text = 'こんにちは'

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは日本語を{language}に訳す翻訳家です。{format_instructions}"),
    HumanMessagePromptTemplate.from_template("以下の文章を翻訳してください。また、合わせて日本語での発音を教えてください。文章: {text}")
])

messages = chat_prompt.format_prompt(language=language, text=text, format_instructions=format_instructions).to_messages()
res = chat.invoke(messages)
print(res.content)

### 放課後ノック111： エージェントを使ってみよう

In [ ]:
!pip install langchain-experimental==0.0.57

In [ ]:
from langchain_community.tools import ShellTool
from langchain.agents import AgentType, initialize_agent

chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0,
                  api_key=os.environ.get("OPENAI_API_KEY"))

shell_tool = ShellTool()
agent = initialize_agent([shell_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

result = agent.invoke("dataディレクトリにあるファイル一覧を教えてください。")
print(result)

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0,
                  api_key=os.environ.get("OPENAI_API_KEY"))

shell_tool = ShellTool()
agent = initialize_agent([shell_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.invoke("dataディレクトリにあるファイル一覧を教えてください。")
print(result)

### 放課後ノック112： 2つのツールを使ってエージェントを高度化しよう

In [ ]:
from langchain_experimental.utilities import PythonREPL
from langchain.agents import Tool

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
agent = initialize_agent([repl_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.invoke("2の4乗を計算してください。")
print(result)

In [ ]:
agent = initialize_agent([shell_tool, repl_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.invoke("dataディレクトリにあるファイルのデータカラムを教えてください。")
print(result)

### 放課後ノック113： LLMを用いた簡易的なデータ分析をやってみよう

In [ ]:
agent = initialize_agent([shell_tool, repl_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.invoke("「data/tbl_order_202004.csv」ファイルの分析を行ってください。統計的な代表値をCSVファイルとして出力してください。")
print(result)

In [ ]:
agent = initialize_agent([shell_tool, repl_tool], chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.invoke("「data/tbl_order_202004.csv」ファイルの分析を行ってください。グラフを１つ作成してpngファイルとして出力してもらえますか。")
print(result)

### 放課後ノック114： PandasのDataFrameを扱えるエージェントを使ってみよう

In [ ]:
import pandas as pd

data = pd.read_csv('data/ml_base_data.csv')
print(len(data))
data.head(5)

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0,
                  api_key=os.environ.get("OPENAI_API_KEY"))

agent = create_pandas_dataframe_agent(chat, data, verbose=True)

In [ ]:
agent.invoke("データの件数は何件ですか")

### 放課後ノック115： エージェントでデータの概要を把握しよう

In [ ]:
agent.invoke("どんなデータなのか説明してください。必ず日本語で説明してください。")

In [ ]:
agent.invoke("このデータに欠損値が含まれているか確認してください。必ず日本語で説明してください。")

### 放課後ノック116： エージェントと一緒に基本的な数字やグラフを押さえよう

In [ ]:
agent.invoke("このデータの統計的な傾向を日本語で説明してください。")

In [ ]:
agent.invoke("このデータ内の「注文数」のデータ分布をグラフで作成してください。")

### 放課後ノック117： エージェントと一緒に分析してみよう

In [ ]:
agent.invoke("「綾瀬店」の「order」の合計値はいくつですか。")

In [ ]:
data.loc[data['store_name'] == '綾瀬店','order'].sum()

In [ ]:
agent.invoke("「order」の値を店舗ごとに合計して、合計値が最も高くなる店舗はどこですか。")

In [ ]:
data_sum = data[['store_name','order']].groupby('store_name',as_index=False).sum()
data_sum.sort_values('order')

### 放課後ノック118： エージェントに分析結果を聞いてみよう

In [ ]:
agent.invoke("このデータから得られるインサイトをいくつか日本語で挙げてください。")

In [ ]:
agent.invoke("このデータから得られる何らかの推論を日本語300字程度の文章でまとめてください。")

### 放課後ノック119： GPT-4によるデータ分析を体験しよう

In [ ]:
gpt_4 = ChatOpenAI(model_name="gpt-4",
                  temperature=0,
                  api_key=os.environ.get("OPENAI_API_KEY"))

agent_4 = create_pandas_dataframe_agent(gpt_4, data, verbose=True)

agent_4.invoke("このデータから得られるインサイトをいくつか日本語で挙げてください。")

In [ ]:
agent_4.invoke("このデータから得られる何らかの推論を日本語300字程度の文章でまとめてください。")

### 放課後ノック120： GPT-4によるデータ可視化を体験しよう

In [ ]:
agent_4.invoke("「order」の合計値を店舗別に集計して、トップ10の店舗に絞り込んで棒グラフで表示してください。")

In [ ]:
!pip install japanize_matplotlib
import japanize_matplotlib

In [ ]:
agent_4.invoke("「order」の合計値を店舗別に集計して、トップ10の店舗に絞り込んで棒グラフで表示してください。")